In [1]:
import scanpy as sc
import pandas as pd
import pickle
import numpy as np
import graph_tool.all as gt

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:39: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:39: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:39: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:40: RuntimeWarning: Error importing draw module, proceeding nevertheless: /usr/lib/python3/dist-packages/cairo/_cairo.cpython-37m-x86_64-linux-gnu.so: undefined symbol: cairo_tee_surface_index
  warnings.warn(msg, RuntimeWarning)


In [2]:
%matplotlib inline
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [3]:
import anndata

In [4]:
n_state = pickle.load( open( "adata_10x/10x_nested_state.pkl", "rb" ) ) 

In [5]:
g = pickle.load( open( "adata_10x/10x_graph_g.pkl", "rb" )) #ho gia il grafo caricato

In [6]:
adata=sc.read('adata_10x/adata_sc_10x.h5ad')
adata

AnnData object with n_obs × n_vars = 860 × 1596 
    obs: 'n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase', 'leiden'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'leiden', 'leiden_colors', 'neighbors', 'pca'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [7]:
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')

In [8]:
df_significativo= pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(20) #### df con i 20 geni significativi per gruppo

In [9]:
cell_annotation= pd.read_table('../sc_mixology/data/csv/sc_10x_5cl.metadata.csv.gz', compression="gzip", sep=",", index_col=0)

In [10]:
adata.obs.loc[:, "cell_line"] = cell_annotation.loc[adata.obs_names, "cell_line"]
adata.obs.columns

Index(['n_genes', 'percent_mito', 'n_counts', 'S_score', 'G2M_score', 'phase',
       'leiden', 'cell_line'],
      dtype='object')

# 1) dopo mcmc_sweeps

### a) state con 5 livelli(corretto)
##### a1) primo tentativo

In [11]:
bs = n_state.get_bs()                   
bs += [np.zeros(1)] * (10 - len(bs))

state = n_state.copy(bs=bs, sampling=True)
state.mcmc_sweep(niter=1000)

(35.94233505310973, 951410, 297584)

In [12]:
state

<NestedBlockState object, with base <BlockState object with 64 blocks (63 nonempty), for graph <Graph object, undirected, with 860 vertices and 5996 edges at 0x7f645d99b250>, at 0x7f645c13f150>, and 10 levels of sizes [(860, 63), (63, 16), (16, 5), (5, 3), (3, 2), (2, 2), (2, 1), (1, 1), (1, 1), (1, 1)] at 0x7f645c13f0d0>

In [16]:
pickle.dump(state, open('adata_10x/10x_collect_info/10x_state_mcmc5.pkl', 'wb'), 2)

In [28]:
state5=pickle.load(open('adata_10x/10x_collect_info/10x_state_mcmc5.pkl', 'rb'))

In [16]:
adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(state5.project_level(2).get_blocks()))
sc.pl.umap(adata, color=["NSBM_test", 'cell_line'], legend_loc='on data')
pd.crosstab(adata.obs.NSBM_test, adata.obs.cell_line)

cell_line,A549,H838,H1975,H2228,HCC827
NSBM_test,,,,,
0,0,0,0,295,0
1,0,0,127,0,0
2,0,176,7,0,0
3,1,0,4,1,188
4,61,0,0,0,0


##### a2) secondo tentativo

In [19]:
bs = n_state.get_bs()                   
bs += [np.zeros(1)] * (10 - len(bs))

state = n_state.copy(bs=bs, sampling=True)
state.mcmc_sweep(niter=1000)

(12.37174843840085, 950989, 296597)

In [20]:
state

<NestedBlockState object, with base <BlockState object with 64 blocks (63 nonempty), for graph <Graph object, undirected, with 860 vertices and 5996 edges at 0x7f645d99b250>, at 0x7f6455356110>, and 10 levels of sizes [(860, 63), (63, 15), (15, 5), (5, 2), (2, 2), (2, 2), (2, 1), (1, 1), (1, 1), (1, 1)] at 0x7f6455356050>

In [21]:
adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(state.project_level(2).get_blocks()))
sc.pl.umap(adata, color=["NSBM_test", 'cell_line'], legend_loc='on data')
pd.crosstab(adata.obs.NSBM_test, adata.obs.cell_line)

cell_line,A549,H838,H1975,H2228,HCC827
NSBM_test,,,,,
1,0,0,127,0,0
2,0,176,7,0,0
3,0,0,0,295,0
4,1,0,4,1,188
5,61,0,0,0,0


### b) state con 4 livelli(non corretto)

In [22]:
state = gt.minimize_nested_blockmodel_dl(g, deg_corr=False)
state

<NestedBlockState object, with base <BlockState object with 57 blocks (57 nonempty), for graph <Graph object, undirected, with 860 vertices and 5976 edges at 0x7f646770c790>, at 0x7f6455517190>, and 5 levels of sizes [(860, 57), (57, 13), (13, 4), (4, 2), (2, 1)] at 0x7f64554adb50>

In [23]:
bs = n_state.get_bs()                   
bs += [np.zeros(1)] * (10 - len(bs))

state = n_state.copy(bs=bs, sampling=True)
state.mcmc_sweep(niter=1000)

(13.63567502509273, 949963, 295781)

In [24]:
pickle.dump(state, open('adata_10x/10x_collect_info/10x_state_mcmc4.pkl', 'wb'), 2)

In [11]:
state4=pickle.load(open('adata_10x/10x_collect_info/10x_state_mcmc4.pkl', 'rb'))

In [12]:
state4

<NestedBlockState object, with base <BlockState object with 63 blocks (63 nonempty), for graph <Graph object, undirected, with 860 vertices and 5996 edges at 0x7f2e43edd750>, at 0x7f2e5e7d6850>, and 10 levels of sizes [(860, 63), (63, 17), (17, 4), (4, 3), (3, 2), (2, 2), (2, 1), (1, 1), (1, 1), (1, 1)] at 0x7f2e43edd250>

In [13]:
adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(state4.project_level(2).get_blocks()))
sc.pl.umap(adata, color=["NSBM_test", 'cell_line'], legend_loc='on data')
pd.crosstab(adata.obs.NSBM_test, adata.obs.cell_line)

... storing 'cell_line' as categorical


cell_line,A549,H838,H1975,H2228,HCC827
NSBM_test,,,,,
0,1,0,4,1,188
1,0,0,127,0,0
2,61,176,7,0,0
4,0,0,0,295,0


# 2) collect info

### a) collect info state5 (force niter 100000)

In [17]:
h = [np.zeros(g.num_vertices() + 1) for s in state4.get_levels()]
pv = [None] * len(state4.get_levels())


def collect_info(s):
    for l,sl in enumerate(s.get_levels()):
        B=sl.get_nonempty_B()
        h[l][B] +=1
        global pv
        pv=[sl.collect_vertex_marginals(pv[l]) for l, sl in enumerate(s.get_levels())]

In [18]:
gt.mcmc_equilibrate(state5, force_niter=10000, mcmc_args=dict(niter=10),
                    callback=collect_info)

(19570.282142472726, 96497339, 30591822)

In [19]:
adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(state5.project_level(3).get_blocks()))
sc.pl.umap(adata, color=["NSBM_test", 'cell_line'], legend_loc='on data')
pd.crosstab(adata.obs.NSBM_test, adata.obs.cell_line)

cell_line,A549,H838,H1975,H2228,HCC827
NSBM_test,,,,,
0,0,176,7,0,0
5,0,0,0,295,0
6,61,0,71,0,0
8,1,0,60,1,188


In [20]:
pickle.dump(state5, open('adata_10x/10x_collect_info/10x_equilibrate_state_mcmc5.pkl', 'wb'), 2)
pickle.dump(h, open('adata_10x/10x_collect_info/10x_h_state_mcmc5.pkl', 'wb'), 2)
pickle.dump(pv, open('adata_10x/10x_collect_info/10x_pv_state_mcmc5.pkl', 'wb'), 2)

In [21]:
estate5=pickle.load(open('adata_10x/10x_collect_info/10x_equilibrate_state_mcmc5.pkl', 'rb'))
h=pickle.load(open('adata_10x/10x_collect_info/10x_h_state_mcmc5.pkl', 'rb'))
pv=pickle.load(open('adata_10x/10x_collect_info/10x_pv_state_mcmc5.pkl', 'rb'))

In [29]:
state5

<NestedBlockState object, with base <BlockState object with 63 blocks (63 nonempty), for graph <Graph object, undirected, with 860 vertices and 5996 edges at 0x7f2e5cd20790>, at 0x7f2e5cd21ad0>, and 10 levels of sizes [(860, 63), (63, 16), (16, 5), (5, 3), (3, 2), (2, 2), (2, 1), (1, 1), (1, 1), (1, 1)] at 0x7f2e5cd20750>

In [30]:
estate5

<NestedBlockState object, with base <BlockState object with 67 blocks (66 nonempty), for graph <Graph object, undirected, with 860 vertices and 5996 edges at 0x7f2e5dc59ad0>, at 0x7f2e5cf86190>, and 10 levels of sizes [(860, 66), (66, 16), (16, 7), (7, 4), (4, 4), (4, 3), (3, 3), (3, 2), (2, 2), (2, 2)] at 0x7f2e5dc594d0>

### b) collect info state 4: mcmc niter 10000, ma d=0.1

In [32]:
state4

<NestedBlockState object, with base <BlockState object with 63 blocks (63 nonempty), for graph <Graph object, undirected, with 860 vertices and 5996 edges at 0x7f2e43edd750>, at 0x7f2e5e7d6850>, and 10 levels of sizes [(860, 63), (63, 17), (17, 4), (4, 3), (3, 2), (2, 2), (2, 1), (1, 1), (1, 1), (1, 1)] at 0x7f2e43edd250>

In [34]:
state_test

<NestedBlockState object, with base <BlockState object with 65 blocks (63 nonempty), for graph <Graph object, undirected, with 860 vertices and 5996 edges at 0x7f2e43edd750>, at 0x7f2e5cd26a10>, and 10 levels of sizes [(860, 63), (63, 16), (16, 5), (5, 2), (2, 2), (2, 2), (2, 2), (2, 1), (1, 1), (1, 1)] at 0x7f2e5cd26990>

In [35]:
bs = state4.get_bs()
bs += [np.zeros(1)] * (10 - len(bs))
state_test = state4.copy(bs=bs)
gt.mcmc_equilibrate(state_test,  mcmc_args=dict(niter=10000, d=0.1), callback=collect_info)

ValueError: Received orphaned property map

In [ ]:
state_test

In [ ]:
%matplotlib inline

In [ ]:
adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(state_test.project_level(2).get_blocks()))
sc.pl.umap(adata, color=["NSBM_test", 'cell_line'], legend_loc='on data')
pd.crosstab(adata.obs.NSBM_test, adata.obs.cell_line)

In [ ]:
pickle.dump(state_test, open('adata_10x/10x_collect_info/10x_equilibrate_state_mcmc4.pkl', 'wb'), 2)
pickle.dump(h, open('adata_10x/10x_collect_info/10x_h_state_mcmc4.pkl', 'wb'), 2)
pickle.dump(pv, open('adata_10x/10x_collect_info/10x_pv_state_mcmc4.pkl', 'wb'), 2)

In [ ]:
estate4=pickle.load(open('adata_10x/10x_collect_info/10x_equilibrate_state_mcmc4.pkl', 'rb'))
h=pickle.load(open('adata_10x/10x_collect_info/10x_h_state_mcmc4.pkl', 'rb'))
pv=pickle.load(open('adata_10x/10x_collect_info/10x_pv_state_mcmc4.pkl', 'rb'))

### c) collect info state 4: force niter 10000, mcmc niter 1, d=0.1

In [24]:
bs = state4.get_bs()

state_test = state4.copy(bs=bs)
gt.mcmc_equilibrate(state_test, force_niter=10000, mcmc_args=dict(niter=1, d=0.1), callback=collect_info)

ValueError: Received orphaned property map

# 3) marginals

In [99]:
marginals=[]
ml = []
for i in range(860):
    lista=[]
    ml.append(len(pv[3][i]))
    for j in pv[3][i]:
        if j!=0:
            lista.append(j)
    marginals.append(lista)

In [100]:
len(ml)

860

In [101]:
marginals

[[12130, 11920, 11630, 9140, 9980, 12150, 10850, 9840, 12350],
 [12600, 11130, 11910, 10160, 11470, 11560, 11150, 10610, 9400],
 [12750, 11330, 12310, 11630, 10110, 11940, 11700, 8880, 9340],
 [15140, 12700, 10530, 10960, 11120, 10520, 10930, 9250, 8840],
 [14490, 9340, 13110, 10400, 9750, 11760, 12110, 9000, 10030],
 [16060, 12480, 9790, 11200, 9340, 11980, 9900, 9000, 10240],
 [10730, 13970, 12720, 10140, 10330, 11650, 10300, 8930, 11190],
 [11330, 12270, 11270, 8850, 12170, 13560, 11710, 9940, 8810],
 [13240, 12410, 12870, 10550, 10330, 10170, 10240, 10360, 9480],
 [15090, 10270, 11480, 10150, 9980, 11610, 11740, 7990, 11340],
 [16260, 8750, 10000, 9740, 10470, 10320, 10140, 8770, 9700],
 [10160, 8050, 9780, 8160, 7830, 13520, 10470, 10230, 10190],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []

In [74]:
s1.collect_vertex_marginals?

In [85]:
p = pv[0]

In [91]:
[x for x in p]

AttributeError: 'NoneType' object has no attribute 'vertices'

# test

In [11]:
bs = n_state.get_bs()                   
bs += [np.zeros(1)] * (10 - len(bs))

state = n_state.copy(bs=bs, sampling=True)
state.multiflip_mcmc_sweep(niter=10000)

(17.470983280579212, 7460162, 103000)

In [12]:
adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(state.project_level(2).get_blocks()))
sc.pl.umap(adata, color=["NSBM_test", 'cell_line'], legend_loc='on data')
pd.crosstab(adata.obs.NSBM_test, adata.obs.cell_line)

... storing 'cell_line' as categorical


cell_line,A549,H838,H1975,H2228,HCC827
NSBM_test,,,,,
0,61,0,71,0,0
3,1,0,4,1,188
5,0,0,56,0,0
6,0,0,0,295,0
8,0,176,7,0,0


In [14]:
pickle.dump(state, open('adata_10x/10x_collect_info/10x_state_mcmc4.pkl', 'wb'), 2)

In [15]:
state=pickle.load(open('adata_10x/10x_collect_info/10x_state_mcmc4.pkl', 'rb'))

In [18]:
gt.mcmc_equilibrate(state, epsilon=0.001,multiflip=True, mcmc_args=dict(niter=10))

(19584.684766180744, 36019864, 643081)

In [19]:
h = [np.zeros(g.num_vertices() + 1) for s in state.get_levels()]

def collect_info(s):
    for l,sl in enumerate(s.get_levels()):
        B=sl.get_nonempty_B()
        h[l][B] +=1

In [23]:
gt.mcmc_equilibrate(state,force_niter=10000,epsilon=0.001, mcmc_args=dict(niter=10),callback=collect_info)

(19541.89243874788, 97382190, 28534977)

In [24]:
for l in range (0,len(bs)):
    idxs= np.where(h[l] > 0)[0]
    v = h[l][idxs]
    figure(l)
    bar(idxs, v)

In [25]:
adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(state.project_level(2).get_blocks()))
sc.pl.umap(adata, color=["NSBM_test", 'cell_line'], legend_loc='on data')
pd.crosstab(adata.obs.NSBM_test, adata.obs.cell_line)

cell_line,A549,H838,H1975,H2228,HCC827
NSBM_test,,,,,
1,0,176,7,0,0
2,0,0,3,0,153
3,0,0,127,0,0
5,0,0,0,205,0
9,61,0,0,0,0
10,1,0,1,1,35
12,0,0,0,90,0


In [26]:
pickle.dump(state, open('adata_10x/10x_collect_info/10x_equilibrate_state_mcmc4.pkl', 'wb'), 2)
pickle.dump(h, open('adata_10x/10x_collect_info/10x_h_state_mcmc4.pkl', 'wb'), 2)

In [27]:
estate=pickle.load(open('adata_10x/10x_collect_info/10x_equilibrate_state_mcmc4.pkl', 'rb'))
h=pickle.load(open('adata_10x/10x_collect_info/10x_h_state_mcmc4.pkl', 'rb'))

In [28]:
state=pickle.load(open('adata_10x/10x_collect_info/10x_state_mcmc4.pkl', 'rb'))

In [29]:
h = [np.zeros(g.num_vertices() + 1) for s in state.get_levels()]

def collect_info(s):
    for l,sl in enumerate(s.get_levels()):
        B=sl.get_nonempty_B()
        h[l][B] +=1

In [30]:
gt.mcmc_equilibrate(state, epsilon=0.001,multiflip=True, mcmc_args=dict(niter=10),callback=collect_info)

(19586.591997283074, 45158612, 792395)

In [31]:
for l in range (0,len(bs)):
    idxs= np.where(h[l] > 0)[0]
    v = h[l][idxs]
    figure(l)
    bar(idxs, v)

In [32]:
adata.obs.loc[:, 'NSBM_test'] = pd.Categorical(list(state.project_level(2).get_blocks()))
sc.pl.umap(adata, color=["NSBM_test", 'cell_line'], legend_loc='on data')
pd.crosstab(adata.obs.NSBM_test, adata.obs.cell_line)

cell_line,A549,H838,H1975,H2228,HCC827
NSBM_test,,,,,
0,0,0,127,0,0
2,0,0,0,156,0
3,1,0,4,1,188
4,0,176,7,0,0
6,0,0,0,139,0
10,61,0,0,0,0
